# Photo Analyzer
- Author: Andrew Eng
- Base Code: https://github.com/dingkeyan93/Intrinsic-Image-Popularity
- Fork: https://github.com/acklab-ae/Intrinsic-Image-Popularity

In [69]:
import argparse
import torch
import torchvision.models
import torchvision.transforms as transforms
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os

# Score photo by popularity

The following code was modified to take in a folder of pictures and process them vs. doing it one by one.

In [70]:
def prepare_image(image):

    if image.mode != 'RGB':
        image = image.convert("RGB")
    Transform = transforms.Compose([
            transforms.Resize([224,224]),      
            transforms.ToTensor(),
            ])
    image = Transform(image)   
    image = image.unsqueeze(0)
    return image.to(device)

def predict(image, model, file, log):
    file = file
    image = prepare_image(image)
    with torch.no_grad():
        preds = model(image)
    score = preds.item()
    with open(log,'a') as f:
        f.write(f'\n{file},{round(score,2)}')
    f.close()
    
def process(x, y, log):
    path = x
    file = y
    image_path = path + file
    #print(f'{file}')
    image = Image.open(image_path)
    model = torchvision.models.resnet50()
    # model.avgpool = nn.AdaptiveAvgPool2d(1) # for any size of the input
    model.fc = torch.nn.Linear(in_features=2048, out_features=1)
    model.load_state_dict(torch.load('model/model-resnet50.pth', map_location=device)) 
    model.eval().to(device)
    predict(image, model, file, log)

# Analyze folder of pictures

In [71]:
image_location = 'test2018/'
log = 'analysis.log'

with open('test.log','w') as f:
    f.write('image,score')
f.close()

list_of_files = os.listdir(image_location)
count = 0

# TQDM is used as a progress bar for image processing
from tqdm import tqdm
import time

count = 0
for i in tqdm(list_of_files):
    process(image_location,list_of_files[count],log)
    count +=1

100%|██████████| 124/124 [03:27<00:00,  1.68s/it]


# Pandas DataFrame for Image Analysis

In [72]:
import pandas as pd

In [73]:
df = pd.read_csv('analysis.log')

In [75]:
df.sort_values(by=['score'], ascending=False)

,image,score
255,2018-08-28 11.40.09.jpg,5.30
131,2018-08-28 11.40.09.jpg,5.30
245,2018-08-15 14.16.19.jpg,5.23
121,2018-08-15 14.16.19.jpg,5.23
1,4.jpg,5.22
171,2018-08-28 11.39.57.jpg,5.15
47,2018-08-28 11.39.57.jpg,5.15
201,2018-08-28 11.35.02.jpg,5.06
77,2018-08-28 11.35.02.jpg,5.06
13,2018-08-28 11.39.59.jpg,5.00


# Facial Recognition

# Background Object Classification

# Location Recognition